In [14]:
from lightgbm import LGBMClassifier
import pandas as pd


In [15]:
train = pd.read_csv("data/train_pre_processing_true_false.csv")
test = pd.read_csv("data/test_pre_processing_true_false.csv")

x_train, y_train = train.select_dtypes(include=['float64','int64','bool']).iloc[:,:-1],train.iloc[:,-1]
x_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_train.columns]
x_test = test.drop(columns=['id']).select_dtypes(include=['float64','int64','bool'])
x_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_test.columns]



In [16]:
preds_dict = {}
rango = 20
for i in range(42+rango,len(x_train.columns),rango):
    
    light_model = LGBMClassifier(random_state=1)
    train_emsemble = x_train.iloc[:,range(0,42)].join(x_train.iloc[:,range(i-rango,i)])
    light_model.fit(train_emsemble, y_train)
    test_emsemble = x_test.iloc[:,range(0,42)].join(x_test.iloc[:,range(i-rango,i)])
    preds = light_model.predict(test_emsemble)
    preds_dict['{} to {}'.format(i-rango,i)] = preds
    
preds = pd.DataFrame(preds_dict).transpose().sum().transform(lambda x: 1 if x >= (len(preds_dict)/2) else 0)


In [17]:
submit = test['id'].to_frame()
submit['target'] = preds
submit.to_csv('data/submit_kaggle_MV', index=False)